In [19]:
# ---------------------------------Load Odrive----------------------------------------------------

In [461]:
"""
Example usage of the ODrive python library to monitor and control ODrive devices
"""

from __future__ import print_function

import odrive
from odrive.enums import *
import time
import math

# Find a connected ODrive (this will block until you connect one)
print("finding an odrive...")
my_drive = odrive.find_any()

# Find an ODrive that is connected on the serial port /dev/ttyUSB0
#my_drive = odrive.find_any("serial:/dev/ttyUSB0")

finding an odrive...


In [247]:
# -------------------------------------Reboot-------------------------------------------------------

In [ ]:
my_drive.reboot()

In [57]:
# ---------------------------------Configuration Values----------------------------------------------

In [462]:
my_drive.axis0.controller.config

control_mode = 3 (int)
pos_gain = 5.0 (float)
vel_gain = 0.0003000000142492354 (float)
vel_integrator_gain = 0.0 (float)
vel_limit = 20000.0 (float)

In [155]:
my_drive.axis0.controller.config.control_mode = 3 
my_drive.axis0.controller.config.pos_gain = 5.0 
my_drive.axis0.controller.config.vel_gain = 0.0003000000142492354 
my_drive.axis0.controller.config.vel_integrator_gain = 0.0
my_drive.axis0.controller.config.vel_limit = 20000.0 

In [39]:
my_drive.axis0.controller.config.pos_gain = 60.0

In [14]:
my_drive.axis0.controller.config.vel_gain = 0.0006000000142492354

In [167]:
my_drive.axis0.controller.config.vel_integrator_gain = 0.00

In [102]:
my_drive.axis1.controller.config.pos_gain = 60.0

In [103]:
my_drive.axis1.controller.config.vel_gain = 0.0006000000142492354

In [184]:
my_drive.axis1.controller.config.vel_integrator_gain = 0.00

In [161]:
my_drive.axis0.motor.config

pre_calibrated = False (bool)
pole_pairs = 7 (int)
calibration_current = 8.0 (float)
resistance_calib_max_voltage = 2.0 (float)
phase_inductance = 0.0 (float)
phase_resistance = 0.0 (float)
direction = -1 (int)
motor_type = 2 (int)
current_lim = 20.0 (float)
requested_current_range = 60.0 (float)

In [156]:
my_drive.axis0.motor.config.pre_calibrated = False 
my_drive.axis0.motor.config.pole_pairs = 7 
my_drive.axis0.motor.config.calibration_current = 8.0 
my_drive.axis0.motor.config.resistance_calib_max_voltage = 2.0 
my_drive.axis0.motor.config.phase_inductance = 0.0
my_drive.axis0.motor.config.phase_resistance = 0.0 
my_drive.axis0.motor.config.direction = 1 
my_drive.axis0.motor.config.motor_type = 2 
my_drive.axis0.motor.config.current_lim = 20.0 
my_drive.axis0.motor.config.requested_current_range = 60.0 

In [162]:
my_drive.axis0.encoder.config

mode = 0 (int)
use_index = True (bool)
pre_calibrated = False (bool)
idx_search_speed = 10.0 (float)
cpr = 8192 (int)
offset = 28711 (int)
offset_float = 1.1356782913208008 (float)
calib_range = 0.019999999552965164 (float)

In [157]:
my_drive.axis0.encoder.config.mode = 0 
my_drive.axis0.encoder.config.use_index = False
my_drive.axis0.encoder.config.pre_calibrated = False
my_drive.axis0.encoder.config.idx_search_speed = 10.0 
my_drive.axis0.encoder.config.cpr = 8192
my_drive.axis0.encoder.config.offset = -4392 
my_drive.axis0.encoder.config.offset_float = -0.030671894550323486 
my_drive.axis0.encoder.config.calib_range = 0.019999999552965164

In [157]:
my_drive.axis1.controller.config

control_mode = 3 (int)
pos_gain = 5.0 (float)
vel_gain = 0.0003000000142492354 (float)
vel_integrator_gain = 0.0 (float)
vel_limit = 20000.0 (float)
vel_limit_tolerance = 1.2000000476837158 (float)
vel_ramp_rate = 10000.0 (float)
setpoints_in_cpr = False (bool)

In [83]:
my_drive.axis1.controller.config.control_mode = 3 
my_drive.axis1.controller.config.pos_gain = 5.0 
my_drive.axis1.controller.config.vel_gain = 0.0003000000142492354 
my_drive.axis1.controller.config.vel_integrator_gain = 0.0
my_drive.axis1.controller.config.vel_limit = 20000.0 

In [454]:
my_drive.axis1.motor.config

pre_calibrated = False (bool)
pole_pairs = 7 (int)
calibration_current = 5.0 (float)
resistance_calib_max_voltage = 2.0 (float)
phase_inductance = 0.0 (float)
phase_resistance = 0.0 (float)
direction = -1 (int)
motor_type = 2 (int)
current_lim = 20.0 (float)
requested_current_range = 60.0 (float)

In [455]:
my_drive.axis1.motor.config.pre_calibrated = False 
my_drive.axis1.motor.config.pole_pairs = 7 
my_drive.axis1.motor.config.calibration_current = 8.0 
my_drive.axis1.motor.config.resistance_calib_max_voltage = 2.0 
my_drive.axis1.motor.config.phase_inductance = 0.0
my_drive.axis1.motor.config.phase_resistance = 0.0 
my_drive.axis1.motor.config.direction = 1 
my_drive.axis1.motor.config.motor_type = 2 
my_drive.axis1.motor.config.current_lim = 20.0 
my_drive.axis1.motor.config.requested_current_range = 60.0 

In [160]:
my_drive.axis1.encoder.config

mode = 0 (int)
use_index = True (bool)
pre_calibrated = True (bool)
idx_search_speed = 10.0 (float)
zero_count_on_find_idx = True (bool)
cpr = 8192 (int)
offset = -4392 (int)
offset_float = -0.030671894550323486 (float)
bandwidth = 1000.0 (float)
calib_range = 0.019999999552965164 (float)

In [85]:
my_drive.axis1.encoder.config.mode = 0 
my_drive.axis1.encoder.config.use_index = False
my_drive.axis1.encoder.config.pre_calibrated = False
my_drive.axis1.encoder.config.idx_search_speed = 10.0 
my_drive.axis1.encoder.config.cpr = 8192
my_drive.axis1.encoder.config.offset = -4392 
my_drive.axis1.encoder.config.offset_float = -0.030671894550323486 
my_drive.axis1.encoder.config.calib_range = 0.019999999552965164

In [80]:
my_drive.vbus_voltage

18.950830459594727

In [ ]:
# ---------------------------------Motor Setup----------------------------------------------------------

In [365]:
# Set up the motor to allow calibration
my_drive.axis0.motor.config.calibration_current = 8.0
my_drive.axis0.motor.config.current_lim = 20.0
my_drive.axis0.motor.config.motor_type = 2

In [415]:
# Set up the motor to allow calibration
my_drive.axis1.motor.config.calibration_current = 5.0
my_drive.axis1.motor.config.current_lim = 20.0
my_drive.axis1.motor.config.motor_type = 2

In [81]:
# ---------------------------------Calibration modes----------------------------------------------------

In [253]:
# Full calibration
print("starting calibration...")
my_drive.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
while my_drive.axis0.current_state != AXIS_STATE_IDLE:
    time.sleep(0.1)

starting calibration...


In [463]:
# Calibration to search index 
my_drive.axis0.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH

In [390]:
# Full calibration
print("starting calibration...")
my_drive.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
while my_drive.axis1.current_state != AXIS_STATE_IDLE:
    time.sleep(0.1)

starting calibration...


In [469]:
# Calibration to search index 
my_drive.axis1.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH

In [12]:
my_drive.axis1.encoder.config.idx_search_speed = 10.0

In [101]:
# ---------------------------------Closed Loop Control-------------------------------------------------

In [34]:
my_drive.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

In [90]:
my_drive.axis0.requested_state = AXIS_STATE_IDLE

In [20]:
my_drive.axis0.controller.config.control_mode = 3

In [44]:
# 1300 makes axis0 at zero position, with pos_gain 60
# -650 makes axis0 at theta1 90 degrees, pos_gain
my_drive.axis0.controller.pos_setpoint = 1300

In [ ]:
my_drive.axis0.controller.vel_setpoint = 1

In [ ]:
my_drive.axis0.controller.current_setpoint = 1

In [90]:
# Default control
my_drive.axis0.controller.config.vel_integrator_gain = 0.0010000000474974513
my_drive.axis0.controller.config.vel_gain = 0.0005000000237487257
my_drive.axis0.controller.config.pos_gain = 20.0

In [91]:
# Spring-like control
my_drive.axis0.controller.config.vel_integrator_gain = 0
my_drive.axis0.controller.config.vel_gain = 0.0003
my_drive.axis0.controller.config.pos_gain = 5

In [94]:
# Testing out control
my_drive.axis0.controller.config.vel_integrator_gain = 0
my_drive.axis0.controller.config.vel_gain = 0.00035
my_drive.axis0.controller.config.pos_gain = 6

In [ ]:
CTRL_MODE_CURRENT_CONTROL

In [87]:
my_drive.axis0.motor.current_control

p_gain = 0.0 (float)
i_gain = nan (float)
v_current_control_integral_d = 0.0 (float)
v_current_control_integral_q = 0.0 (float)
Ibus = 0.0 (float)
final_v_alpha = 0.0 (float)
final_v_beta = 0.0 (float)
Iq_setpoint = 0.0 (float)
Iq_measured = 0.0 (float)
max_allowed_current = 60.75 (float)
overcurrent_trip_level = 67.5 (float)

In [458]:
my_drive.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

In [472]:
my_drive.axis1.requested_state = AXIS_STATE_IDLE

In [24]:
my_drive.axis1.controller.config.control_mode = 3

In [51]:
# 1450 makes axis1 at zero position, with gain 30
# 0 makes axis1 at theta2 at 90, with gain 30
my_drive.axis1.controller.pos_setpoint = 0

In [ ]:
# ---------------------------------Index Saving Setup---------------------------------------------------

In [ ]:
# With this configuration, we only need to do this after each reboot:
# my_drive.axis0.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH

In [195]:
my_drive.axis0.encoder.config.use_index = True

In [196]:
my_drive.axis0.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH

In [197]:
my_drive.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE

In [198]:
my_drive.axis0.encoder.config.pre_calibrated = True

In [199]:
my_drive.axis0.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH

In [200]:
my_drive.axis0.motor.config.pre_calibrated = True

In [201]:
my_drive.save_configuration()

In [473]:
my_drive.axis1.encoder.config.use_index = True

In [475]:
my_drive.axis1.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH

In [476]:
my_drive.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE

In [437]:
my_drive.axis1.encoder.config.pre_calibrated = False

In [381]:
my_drive.axis1.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH

In [438]:
my_drive.axis1.motor.config.pre_calibrated = False

In [102]:
my_drive.save_configuration()

In [71]:
# ---------------------------------Testings-----------------------------------------------------------

In [184]:
my_drive.axis1.encoder.pos_estimate

2.999999761581421

In [185]:
my_drive.axis1.encoder.vel_estimate

0.0

In [68]:
my_drive.axis0.motor.current_control.Iq_measured

0.0

In [ ]:
# ----------------------------------Other features-----------------------------------------------------

In [ ]:
# To read a value, simply read the property
print("Bus voltage is " + str(my_drive.vbus_voltage) + "V")

# Or to change a value, just assign to the property
my_drive.axis0.controller.pos_setpoint = 3.14
print("Position setpoint is " + str(my_drive.axis0.controller.pos_setpoint))

In [41]:
# And this is how function calls are done:
for i in [1,2,3,4,5,6,7,8]:
    print('voltage on GPIO{} is {} Volt'.format(i, my_drive.get_adc_voltage(i)))

voltage on GPIO1 is 0.7508788704872131 Volt
voltage on GPIO2 is 0.6380859017372131 Volt
voltage on GPIO3 is 0.652587890625 Volt
voltage on GPIO4 is 0.701733410358429 Volt
voltage on GPIO5 is 1.4381103515625 Volt
voltage on GPIO6 is nan Volt
voltage on GPIO7 is nan Volt
voltage on GPIO8 is nan Volt


In [42]:
my_drive.GPIO_set_to_analog(my_drive.get_gpio_port_by_pin(1), 1);

AttributeError: 'RemoteObject' object has no attribute 'GPIO_set_to_analog'

In [45]:
my_drive.get_adc_voltage(1)

0.760546863079071

In [43]:
my_drive.get_gpio_port_by_pin(1)

AttributeError: 'RemoteObject' object has no attribute 'get_gpio_port_by_pin'

In [151]:
# A sine wave to test
t0 = time.monotonic()
while True:
    # prev 10000.0
    setpoint = 3000.0 * math.sin((time.monotonic() - t0)*3)
#     print("goto " + str(int(setpoint)))
    #my_drive.axis0.controller.pos_setpoint = setpoint+1300
    my_drive.axis1.controller.pos_setpoint = -(setpoint+1450)
    time.sleep(0.001)

# Some more things you can try:

ChannelBrokenException: 

In [152]:
# Write to a read-only property:
my_drive.vbus_voltage = 11.0  # fails with `AttributeError: can't set attribute`

# Assign an incompatible value:
my_drive.motor0.pos_setpoint = "I like trains"  # fails with `ValueError: could not convert string to float`

AttributeError: Attribute vbus_voltage not found

In [ ]:
import sys
sys.version